# Code Interpreter

OpenAIが提供するAssistant APIのうちの一つの機能．入力されたメッセージに対してPythonのコードを生成し実行し回答を出力する．
- ドキュメント:https://platform.openai.com/docs/assistants/tools/code-interpreter

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/main/nlp/llm_framework/code_interpreter.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install openai

In [1]:
from openai import OpenAI
import os

client = OpenAI()

#os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"


### 入力の設定

In [36]:
instruction = "あなたは数学の専門家です。数学の質問に答えるコードを書いて実行します。"
model = "gpt-4-1106-preview"
content = "21x^2+4x-32=0の解を求めてください。"


In [37]:
# アシスタントの作成
assistant = client.beta.assistants.create(
    instructions = instruction,
    model = model,
    tools=[{"type": "code_interpreter"}],
)

# スレッドの作成
thread = client.beta.threads.create()

# スレッドにメッセージを追加
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = content,
)


In [38]:
# スレッドにメッセージが追加されていることを確認する
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)
for message in messages:
    print(message.role, ":", message.content[0].text.value)


user : 21x^2+4x-32=0の解を求めてください。


### Code Interpreterの実行

In [39]:
# アシスタントにリクエスト
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)


### APIでの実行状況の確認
- in_progress:実行中
- completed:実行完了

In [43]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
print(run.status)


completed


### 出力結果を確認
- statusが「completed」になったら確認する

In [44]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)
for message in messages:
    print(message.role, ":", message.content[0].text.value)



user : 21x^2+4x-32=0の解を求めてください。
assistant : 方程式 \(21x^2+4x-32=0\) の解は \(x = -\frac{4}{3}\) と \(x = \frac{8}{7}\) です。


### 実行状況の確認

In [20]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)
run_steps


SyncCursorPage[RunStep](data=[RunStep(id='step_NjUpiehM1LSkRxU5DyjPWKae', assistant_id='asst_PvvMYkORDjMjH90fkaonjWDR', cancelled_at=None, completed_at=1699456811, created_at=1699456810, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_OdmGWxyWNegCRjULzvkFy9VL', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_s093Ysz4cIj75IUR3vWN0NOv'), type='message_creation'), thread_id='thread_tThAyjP1GyTMjDVS9ODOCQO5', type='message_creation', expires_at=None), RunStep(id='step_4NvqnP4Y2AdxzEtd5jEoETL3', assistant_id='asst_PvvMYkORDjMjH90fkaonjWDR', cancelled_at=None, completed_at=1699456810, created_at=1699456807, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_OdmGWxyWNegCRjULzvkFy9VL', status='completed', step_details=ToolCallsStepDetails(tool_calls=[CodeToolCall(id='call_tKIXjgv2oq9oeusC2CVxDz59', code_interpreter=CodeInterpret